In [20]:
import numpy as np
import pandas
import os
from tabulate import tabulate

In [21]:
def cal_score(arr):
    ans = 0
    for i in range(0,len(arr)):
        for j in range(i+1,len(arr)):
            node1 = i+1
            node2 = j+1
            trueEdge = dict.get((node1,node2),0)
            if arr[i]!=arr[j] and trueEdge == 1:
                ans += 1
    return -ans

In [22]:
# tournament selection
def selection(pop, scores, k=3):
	# first random selection
	selection_ix = np.random.randint(len(pop))
	for ix in np.random.randint(0, len(pop), k-1):
		# check if better (e.g. perform a tournament)
		if scores[ix] < scores[selection_ix]:
			selection_ix = ix
	return pop[selection_ix]

In [23]:
# crossover two parents to create two children
def crossover(p1, p2, r_cross):
	# children are copies of parents by default
	c1, c2 = p1.copy(), p2.copy()
	# check for recombination
	if np.random.rand() < r_cross:
		# select crossover point that is not on the end of the string
		pt = np.random.randint(1, len(p1)-2)
		# perform crossover
		c1 = p1[:pt] + p2[pt:]
		c2 = p2[:pt] + p1[pt:]
	return [c1, c2]

In [24]:
# mutation operator
def mutation(bitstring, r_mut):
	for i in range(len(bitstring)):
		# check for a mutation
		if np.random.rand() < r_mut:
			# flip the bit
			bitstring[i] = 1 - bitstring[i]

In [25]:
def GA(cal_score, n_bits, n_loop, n_pop, r_cross, r_mut):
    pop = [np.random.randint(0, 2, n_bits).tolist() for _ in range(n_pop)]

    best, best_eval = 0, cal_score(pop[0])

    for gen in range(n_loop):
        scores = [cal_score(_) for _ in pop]
        for i in range(n_pop):
            if scores[i] < best_eval:
                best, best_eval = pop[i], scores[i]
                # print(">%d, new best f(%s) = %.3f" % (gen,  pop[i], scores[i]))

        selected = [selection(pop, scores) for _ in range(n_pop)]

        children = list()
        for i in range(0, n_pop, 2):
            par1, par2 = selected[i], selected[i+1]
            for _ in crossover(par1, par2, r_cross):
                mutation(_, r_mut)
                children.append(_)
        pop = children

    return [best, -best_eval]

In [27]:
test_directory = './Test'
for item in os.listdir(test_directory):
    item_path = os.path.join(test_directory, item)
    with open(item_path, 'r', encoding='utf-8') as file:
        _ = file.read()
    _ = _.split()
    n, m, optimalValue = int(_[0]), int(_[1]), int(_[2])

    dict = {}
    for i in range(3,len(_),3):
        dict[(int(_[i]),int(_[i+1]))] = int(_[i+2])
        dict[(int(_[i+1]),int(_[i]))] = int(_[i+2])

    n_pop,n_bits = 100, n
    n_loop = 50
    r_cross = 0.9
    r_mut = 1.0/float(n_bits)

    num_node = []
    num_edge = []
    optimalValues = []
    scores = []

    # GA
    for k in range(30):
        best, score = GA(cal_score, n_bits, n_loop, n_pop, r_cross, r_mut)

        optimalValues.append(optimalValue)
        scores.append(score)
        num_node.append(n)
        num_edge.append(m)

    df = pandas.DataFrame({
        # 'Số đỉnh': num_node,
        # 'Số cạnh': num_edge,
        'Giá trị tìm được': scores,
        'Giá trị chính xác': optimalValues
    })

    print("Test ",item)
    print(tabulate(df, headers='keys', tablefmt='grid'))

    sum_scores = sum(scores)
    average_score = float(sum_scores/30)
    percent = 100*(float(optimalValue-average_score)/optimalValue)
    print("Trung bình giá trị tìm được: ",average_score)
    print("Giá trị chính xác: ",optimalValue)
    print("Delta = ",percent,"%")
    print("\n")

Test  g05_100.0
+----+--------------------+---------------------+
|    |   Giá trị tìm được |   Giá trị chính xác |
+====+====================+=====================+
|  0 |               1402 |                1430 |
+----+--------------------+---------------------+
|  1 |               1391 |                1430 |
+----+--------------------+---------------------+
|  2 |               1383 |                1430 |
+----+--------------------+---------------------+
|  3 |               1398 |                1430 |
+----+--------------------+---------------------+
|  4 |               1395 |                1430 |
+----+--------------------+---------------------+
|  5 |               1410 |                1430 |
+----+--------------------+---------------------+
|  6 |               1387 |                1430 |
+----+--------------------+---------------------+
|  7 |               1401 |                1430 |
+----+--------------------+---------------------+
|  8 |               1396 |       